In [26]:
import pandas as pd
import numpy as np
import pickle
import json


In [27]:
# import matplotlib.pyplot as plt
# from sklearn import datasets, model_selection
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [28]:
with open('D:\study\Spring 2024\cs education\combined_part\calculator\column_order.json','r') as f:
    column_order = json.load(f)
column_order

['Lab 13 PDF [Total Pts: 10 Score] |1883440',
 'Chapter 2 - Checkpoint 4 [Total Pts: 4 Score] |1936680',
 'Lab 6 [Total Pts: 20 Score] |1883423',
 'Quiz 5 [Total Pts: 20 Score] |2009890',
 'Chapter 4 - Checkpoint 9 [Total Pts: 4 Score] |1994622',
 'Chapter 5 - Checkpoint 2 [Total Pts: 4 Score] |1994627',
 'Homework 7 [Total Pts: 40 Score] |1884933',
 'Lab 2 [Total Pts: 20 Score] |1883408',
 'Lab 1 [Total Pts: 20 Score] |1882989',
 'Chapter 3 - Checkpoint 3 [Total Pts: 4 Score] |1938813',
 'Quiz 6 [Total Pts: 20 Score] |2009024',
 'Project 2 [Total Pts: 100 Score] |2023051',
 'Chapter 4 - Checkpoint 2 [Total Pts: 4 Score] |1945791',
 'Chapter 4 - Checkpoint 6 [Total Pts: 4 Score] |1949679',
 'Lab 14 [Total Pts: 20 Score] |1883441',
 'Lab 8 [Total Pts: 20 Score] |1883432',
 'Chapter 4 - Checkpoint 4 [Total Pts: 4 Score] |1949674',
 'Chapter 5 - Checkpoint 1 [Total Pts: 4 Score] |1994626',
 'Chapter 4 - Checkpoint 5 [Total Pts: 4 Score] |1949678',
 'Homework 3 final grade [Total Pts: 40 S

In [29]:
model_filename = f'D:\study\Spring 2024\cs education\combined_part\predictor\model_stored\ model_week_3.pkl'

with open(model_filename, 'rb') as model_file:
    model = pickle.load(model_file)
    print(model.feature_names_in_)

['Chapter 2 - Checkpoint 5 [Total Pts: 4 Score] |1936681'
 'Chapter 2 - Checkpoint 4 [Total Pts: 4 Score] |1936680'
 'Homework 1 Final Grade [Total Pts: 40 Score] |1995600'
 'Chapter 2 - Checkpoint 3 [Total Pts: 4 Score] |1936678'
 'Lab 2 [Total Pts: 20 Score] |1883408'
 'Lab 1 [Total Pts: 20 Score] |1882989']


# Predictor_UI


In [30]:
from tkinter import filedialog
import tkinter as tk
from tkinter import ttk
from tkinter import *

In [38]:
from tkinter import scrolledtext


def select_file():
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        file_path_var.set(file_path) 
        df = pd.read_csv(file_path)
        update_column_selection(df.columns)
        # with open(f'D:\study\Spring 2024\cs education\combined_part\calculator\column_order\ column_order_3.json','r') as f:
        #             column_order = json.load(f)
        #         # column_order
        # df_predictor = df[column_order]
        # # df_predictor = df[column_order]
        # update_column_selection(df_predictor.columns)

def update_column_selection(columns): # to select columns
    column_listbox.delete(0, tk.END)
    for column in columns:
        column_listbox.insert(tk.END, column)
    column_listbox.config(selectmode="multiple")


def grade_set(predtion_points):
    if predtion_points >= 900:
        return "A"
    elif predtion_points >= 800:
        return "B"
    elif predtion_points >= 700:
        return "C"
    elif predtion_points >= 600:
        return "D"
    else:
        return "F"
    
def read_result_file(filepath):
    r2_result = pd.read_csv(filepath)
    return r2_result

def predict_grade():
    week = int(week_number.get())
    file_path = file_path_var.get()
    # selected_indices = column_listbox.curselection()
    # selected_columns = [column_listbox.get(i) for i in selected_indices] # Above two to get selected columns
    
    # for i in file_path.columns:
    #     selected_columns = column_listbox.get(i)

    if not file_path:
        results_textbox.insert(tk.INSERT,"Error: Please select a CSV file.")
        return
    
    # df = pd.read_csv(file_path, usecols=selected_columns)  # to read the csv
    df = pd.read_csv(file_path)  # to read the csv

    # df_predictor = df[column_order]
    # update_column_selection(df_predictor.columns)
    try:

        with open(f'D:\study\Spring 2024\cs education\combined_part\calculator\column_order\ column_order_{week}.json','r') as f:
            column_order = json.load(f)
                # column_order
        df_predictor = df[column_order]
        # df_predictor = df[column_order]
        update_column_selection(df_predictor.columns)
        results_textbox.delete(1.0,END)

        data = df_predictor
        # with open(f'D:\study\Spring 2024\cs education\combined_part\calculator\column_order\ column_order_{week}.json','r') as f:
        #     column_order = json.load(f)
        # # column_order
        # df_predictor = df[column_order]
        # update_column_selection(df_predictor.columns)
        model_filename = f'D:\study\Spring 2024\cs education\combined_part\predictor\model_stored\ model_week_{week}.pkl'

        with open(model_filename, 'rb') as model_file:
            model = pickle.load(model_file)
        # print(list(zip(model_filename.coef_,model_filename.features)))
        
        predictions = model.predict(data)
        predictions_df = pd.DataFrame(predictions, columns=['Predicted Points'])
        # print(predictions)
        # print(predictions_df)  # check the grade part
        new_pre_df = predictions_df
        new_pre_df['Predicted Grade'] = new_pre_df['Predicted Points'].apply(grade_set)
    
        predictions_file_path = f'D:\study\Spring 2024\cs education\combined_part\predictor\predict_points_grades_results\ week_{week}_predictions_result.csv'
        new_pre_df.to_csv(predictions_file_path, index=False)

        show_r2score()
        df_content = new_pre_df.to_string(index = False)
        results_textbox.insert(tk.INSERT,df_content)

        # result_var.set(f"Predicted Points: {predictions}")
    except Exception as e:
        results_textbox.delete("1.0", tk.END)
        results_textbox.insert(tk.INSERT, f"Error: {str(e)}")

def show_r2score():
    week = int(week_number.get())
    r2_score = r2_result.loc[r2_result['Weeks'] == f"Week{week}",'R2-Score'].values[0]
    result_label.config(text = f"R2-Score for Week{week}: {r2_score}")

def week_number_changed(*args):
    if week_number.get().isdigit():
        predict_grade()

result_file_path = 'D:\study\Spring 2024\cs education\combined_part\predictor\ R2_result.csv'
r2_result = read_result_file(result_file_path)

root = tk.Tk()
root.title("The Grade Predictor")

mainframe = ttk.Frame(root, padding="12 12 12 12")
mainframe.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

week_number = tk.StringVar()
# week_number.trace_add("w",week_number_changed)
# result_var = tk.StringVar()

week_number_label = ttk.Label(mainframe, text="Week Number:")
week_number_label.grid(column=0, row=0, sticky=(tk.W,tk.E))
week_number_entry = ttk.Entry(mainframe, width=7, textvariable=week_number)
week_number_entry.grid(column=1, row=0, sticky=(tk.W))

file_path_var = tk.StringVar()
select_file_button = ttk.Button(mainframe, text="Select CSV File", command=select_file)
select_file_button.grid(column=2, row=1)
file_path_entry = ttk.Entry(mainframe, width=40, textvariable=file_path_var, state="readonly")
file_path_entry.grid(column=0, row=2, columnspan=2)

column_listbox = tk.Listbox(mainframe)
column_listbox.grid(column=0, row=3, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))  # Above two to select columns (new)

predict_button = ttk.Button(mainframe, text="Predict Final Points", command=predict_grade)
predict_button.grid(column=2, row=2)

result_label = tk.Label(root, text="")
result_label.grid(column=0, row=4, columnspan=3, sticky=tk.W)

results_textbox = scrolledtext.ScrolledText(mainframe, width=40, height=10)
results_textbox.grid(column=0, row=5, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S))


root.mainloop()


In [32]:
import os
import re

In [39]:
folder1_path = 'D:\study\Spring 2024\cs education\combined_part\predictor\csv_chosen'
folder2_path = 'D:\study\Spring 2024\cs education\combined_part\predictor\predict_points_grades_results'
output_folder_path = 'D:\study\Spring 2024\cs education\combined_part\predictor\ final_predictor_results'

os.makedirs(output_folder_path, exist_ok=True)

week_pattern = re.compile(r'week_\d+')

for file1 in os.listdir(folder1_path):
    match = week_pattern.search(file1)
    if match:
        week_part = match.group(0)
        for file2 in os.listdir(folder2_path):
            if week_part in file2:
                file_path1 = os.path.join(folder1_path, file1)
                file_path2 = os.path.join(folder2_path, file2)
                df1 = pd.read_csv(file_path1)
                df2 = pd.read_csv(file_path2)
                
                merged_df = pd.concat([df1, df2], axis=1)
                
                output_file_name = f"final_merged_{week_part}_result.csv"
                output_file_path = os.path.join(output_folder_path, output_file_name)
                merged_df.to_csv(output_file_path, index=False)
                print(f"Merged file saved as: {output_file_name}")
                print(output_file_path)
                break

Merged file saved as: final_merged_week_2_result.csv
D:\study\Spring 2024\cs education\combined_part\predictor\ final_predictor_results\final_merged_week_2_result.csv
Merged file saved as: final_merged_week_3_result.csv
D:\study\Spring 2024\cs education\combined_part\predictor\ final_predictor_results\final_merged_week_3_result.csv
Merged file saved as: final_merged_week_4_result.csv
D:\study\Spring 2024\cs education\combined_part\predictor\ final_predictor_results\final_merged_week_4_result.csv
Merged file saved as: final_merged_week_5_result.csv
D:\study\Spring 2024\cs education\combined_part\predictor\ final_predictor_results\final_merged_week_5_result.csv
